# QuakeMigrate example - Icequake detection at the Rutford Ice Stream, Antarctica

## Overview

This notebook contains an example demonstrating how to run QuakeMigrate for icequake detection.

Here, we detail how to:

1. Calculate a travel-time lookup table for the seismometer network and example grid, using a velocity model with homogeneous P- and S-wave velocities.
2. Run the detect stage, to continuously migrate and stack phase arrival onset functions at each point in the grid, searching for coherent sources of energy in space and time.
3. Run the trigger stage to identify candidate events from the continuous detect output.
4. Run the locate stage to calculate refined locations for these candidate events, and provide a range of additional outputs and plots, including robust location uncertainty estimates and phase picks.

In [ ]:
import pandas as pd
from obspy.core import AttribDict
from pyproj import Proj

from quakemigrate import QuakeScan, Trigger
from quakemigrate.io import Archive, read_stations
from quakemigrate.lut import compute_traveltimes
from quakemigrate.signal.onsets import STALTAOnset
from quakemigrate.signal.pickers import GaussianPicker

%matplotlib inline

In [ ]:
# --- i/o paths ---
station_file = "./inputs/rutford_stations.txt"
data_in = "./inputs/mSEED"
lut_out = "./outputs/lut/icequake.LUT"
run_path = "./outputs/runs"
run_name = "icequake_example"

## 1. Calculate a travel-time lookup table (LUT)

In [ ]:
# --- Read in the station information file ---
stations = read_stations(station_file)

# --- Define the input and grid projections ---
gproj = Proj(proj="lcc", units="km", lon_0=-83.932, lat_0=-78.144, lat_1=-78.1,
             lat_2=-77.9, datum="WGS84", ellps="WGS84", no_defs=True)
cproj = Proj(proj="longlat", datum="WGS84", ellps="WGS84", no_defs=True)

# --- Define the grid specifications ---
# AttribDict behaves like a Python dict, but also has '.'-style access.
grid_spec = AttribDict()
grid_spec.ll_corner = [-84.14853353566141, -78.18825429331356, -0.350]
grid_spec.ur_corner = [-83.71921885073093, -78.10003166259442, 3.550]
grid_spec.node_spacing = [0.1, 0.1, 0.1]
grid_spec.grid_proj = gproj
grid_spec.coord_proj = cproj

# --- Homogeneous LUT generation ---
lut = compute_traveltimes(grid_spec, stations, method="homogeneous",
                          phases=["P", "S"], vp=3.841, vs=1.970, log=True,
                          save_file=lut_out)

## 2. Run the detect stage: continuously migrate phase arrival onset functions through the grid to detect coalescence peaks in space and time

In [ ]:
# --- Read in station file ---
stations = read_stations(station_file)

# --- Create new Archive and set path structure ---
archive = Archive(archive_path=data_in, stations=stations,
                  archive_format="YEAR/JD/*_STATION_*")

# --- Create new Onset ---
onset = STALTAOnset(position="classic", sampling_rate=500)
onset.phases = ["P", "S"]
onset.bandpass_filters = {
    "P": [20, 200, 4],
    "S": [10, 125, 4]}
onset.sta_lta_windows = {
    "P": [0.01, 0.25],
    "S": [0.05, 0.5]}

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, run_path=run_path,
                 run_name=run_name, log=True, loglevel="info")

# --- Set detect parameters ---
scan.timestep = 0.75
# NOTE: please increase the thread-count as your system allows; the
# core migration routines are compiled against OpenMP, and using
# multithreading will ~ linearly speed up the compute time!
scan.threads = 1

# --- Set time period over which to run detect ---
starttime = "2009-01-21T04:00:05.0"
endtime = "2009-01-21T04:00:10.0"

In [ ]:
# --- Run detect ---
scan.detect(starttime, endtime)

## 3. Run the trigger stage: identify individual icequakes from the continuous detect output

In [ ]:
# --- Create new Trigger ---
trig = Trigger(lut, run_path, run_name, log=True, loglevel="info")

# --- Set trigger parameters ---
trig.marginal_window = 0.1
trig.min_event_interval = 0.5
trig.normalise_coalescence = True

# --- Static threshold ---
trig.threshold_method = "static"
trig.static_threshold = 2.75

# --- Run trigger ---
trig.trigger(starttime, endtime, interactive_plot=False)

### Show the trigger summary PDF file
NOTE: this may not display properly for some OS's / browsers (e.g. Safari on MacOS). If all you see is a grey box, please try opening the notebook in Chrome or Firefox, or open the file directly using a PDF viewer.

In [ ]:
# Show the trigger summary pdf file
icequake_trigger_summary_image_fname = "outputs/runs/icequake_example/trigger/summaries/icequake_example_2009_021_Trigger.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_trigger_summary_image_fname, width=800, height=450) # Plot pdf

## 4. Run the locate stage: calculate refined locations and location uncertainty estimates

Note: Here we create a new onset object, using the "centred" STALTAOnset. This produces a more accurate gaussian representation of the phase arrival probability density function, and is less phase-shifted. However, it is much more sensitive to sharp offsets due to instrument spikes etc., and is less flexible in identifying arrivals with different frequency content than the "classic" STALTAOnset, so in general that is the better choice to use for detect().

In [ ]:
# --- Create new Onset ---
onset = STALTAOnset(position="centred", sampling_rate=1000)
onset.phases = ["P", "S"]
onset.bandpass_filters = {
    "P": [20, 200, 4],
    "S": [10, 125, 4]}
onset.sta_lta_windows = {
    "P": [0.01, 0.25],
    "S": [0.05, 0.5]}

# --- Create new PhasePicker ---
picker = GaussianPicker(onset=onset)
picker.plot_picks = True

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, picker=picker,
                 run_path=run_path, run_name=run_name, log=True,
                 loglevel="info")

# --- Set locate parameters ---
scan.marginal_window = 0.1
# NOTE: please increase the thread-count as your system allows; the
# core migration routines are compiled against OpenMP, and using
# multithreading will ~ linearly speed up the compute time!
scan.threads = 1

# Turn on plotting features
scan.plot_event_summary = True

# --- Toggle writing of waveforms ---
scan.write_cut_waveforms = False

In [ ]:
# --- Run locate ---
scan.locate(starttime=starttime, endtime=endtime)

## 5. Some of the key locate outputs

In [ ]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = "./outputs/runs/icequake_example/locate/events/20090121040007152.event"
event_df = pd.read_csv(icequake_event_fname)

event_df

In [ ]:
# Show the .picks file, containing station time picks:
icequake_pick_fname = "outputs/runs/icequake_example/locate/picks/20090121040007152.picks"
pick_df = pd.read_csv(icequake_pick_fname)

pick_df

In [ ]:
# Show the event summary pdf file, containing event origin time and location, a
# plot displaying the 3D coalescence image, and a waveform gather showing the
# fit of the modelled arrival times to the data:
icequake_event_summary_image_fname = "outputs/runs/icequake_example/locate/summaries/icequake_example_20090121040007152_EventSummary.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_event_summary_image_fname, width=800, height=550) # Plot pdf

# NOTE: this may not display properly for some OS's / browsers (e.g. Safari on
# MacOS). If all you see is a grey box, please try opening the notebook in
# Chrome or Firefox, or open the file directly using a PDF viewer.

In [ ]:
# Show an example of a pick summary pdf file, displaying the onset functions
# for P and S phases, the gaussian fit to the onset function within the pick
# window, and the resulting picktime, uncertainty and SNR.
icequake_event_summary_image_fname = "outputs/runs/icequake_example/locate/pick_plots/20090121040007152/20090121040007152_ST10.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_event_summary_image_fname, width=800, height=550) # Plot pdf

# NOTE: this may not display properly for some OS's / browsers (e.g. Safari on
# MacOS). If all you see is a grey box, please try opening the notebook in
# Chrome or Firefox, or open the file directly using a PDF viewer.